In [174]:
from selenium import webdriver
import time
import bs4 as bs
from string import printable
import re
import os
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests,json
import pickle
import numpy as np
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta

In [175]:
df = pd.read_csv("plswork.csv") 


In [176]:
df[['Wins','Losses','Draws']] = df.Record.str.split("-",expand=True,)

In [177]:
df['Wins'] = df['Wins'].astype(int)
df['Losses'] = df['Losses'].astype(int)
df['Draws'] = df['Draws'].astype(int)

In [178]:
df['Win Percentage'] = round(df['Wins']/(df['Wins'] + df['Losses'] + df['Draws']),2)

In [179]:
df = df[~df['Height:'].str.contains("--")]
df = df[~df['Reach:'].str.contains("--")]
df = df[~df['DOB:'].str.contains("--")]
df = df[~df['Str. Acc.:'].str.contains(" ")]

In [180]:
df['Nickname'] = df['Nickname'].fillna('No Name')

In [181]:
len(df.Nickname)

1672

In [182]:
df.head()

,Unnamed: 0,Name,Record,Nickname,Height:,Weight:,Reach:,STANCE:,DOB:,SLpM:,...,SApM:,Str. Def:,TD Avg.:,TD Acc.:,TD Def.:,Sub. Avg.:,Wins,Losses,Draws,Win Percentage
3,3,Shamil Abdurakhimov,20-5-0,Abrek,"6' 3""",235 lbs.,"76""",Orthodox,"Sep 02, 1981",2.45,...,2.45,58%,1.23,24%,47%,0.2,20,5,0,0.80
5,5,Daichi Abe,6-2-0,No Name,"5' 11""",170 lbs.,"71""",Orthodox,"Nov 27, 1991",3.80,...,4.49,56%,0.33,50%,0%,0.0,6,2,0,0.75
6,6,Papy Abedi,9-3-0,Makambo,"5' 11""",185 lbs.,"74""",Southpaw,"Jun 30, 1978",2.80,...,3.15,48%,3.47,57%,50%,1.3,9,3,0,0.75
7,7,Ricardo Abreu,5-3-0,Demente,"5' 11""",185 lbs.,"73""",Orthodox,"Apr 27, 1984",3.79,...,3.98,68%,2.13,42%,100%,0.7,5,3,0,0.62
8,8,Klidson Abreu,15-3-0,White Bear,"6' 0""",205 lbs.,"74""",Orthodox,"Dec 24, 1992",1.97,...,1.93,59%,0.50,12%,75%,0.0,15,3,0,0.83


In [183]:
df.columns

Index(['Unnamed: 0', 'Name', 'Record', 'Nickname', 'Height:', 'Weight:',
       'Reach:', 'STANCE:', 'DOB:', 'SLpM:', 'Str. Acc.:', 'SApM:',
       'Str. Def:', 'TD Avg.:', 'TD Acc.:', 'TD Def.:', 'Sub. Avg.:', 'Wins',
       'Losses', 'Draws', 'Win Percentage'],
      dtype='object')

In [184]:
df[['Feet','Inches']] = df['Height:'].str.split("'",expand=True)
df['Reach:'] = df['Reach:'].str.slice(0,-1).astype(int)
df['Weight:'] = df['Weight:'].str.slice(0,-5).astype(int)


In [185]:
df['Inches'] = df['Inches'].str.slice(0,-1)
df['Feet'] = df['Feet'].astype(int)
df['Inches'] = df['Inches'].astype(int)
df['Inches'] = df['Inches']/12
df['Height:'] = round(df['Feet'] + df['Inches'],2)

In [186]:
df = df.drop(['Feet', 'Inches', 'Record','Unnamed: 0','STANCE:'], axis=1)
df = df.rename(columns = {'DOB:':"Age"})

In [187]:
df['Age'] = pd.to_datetime(df['Age'],format='%b %d, %Y')

In [188]:
now = dt.today()
df['Age'] = df['Age'].where(df['Age'] < now, df['Age'] -  np.timedelta64(100, 'Y'))   
df['Age'] = (now - df['Age']).astype('<m8[Y]') 

In [189]:
df.dtypes

Name               object
Nickname           object
Height:           float64
Weight:             int32
Reach:              int32
Age               float64
SLpM:             float64
Str. Acc.:         object
SApM:             float64
Str. Def:          object
TD Avg.:          float64
TD Acc.:           object
TD Def.:           object
Sub. Avg.:        float64
Wins                int32
Losses              int32
Draws               int32
Win Percentage    float64
dtype: object

In [190]:
df["Str. Acc.:"] = df["Str. Acc.:"].replace({'%':''}, regex = True)
df["Str. Def:"] = df["Str. Def:"].replace({'%':''}, regex = True)
df["TD Acc.:"] = df["TD Acc.:"].replace({'%':''}, regex = True)
df["TD Def.:"] = df["TD Def.:"].replace({'%':''}, regex = True)

In [191]:
df

,Name,Nickname,Height:,Weight:,Reach:,Age,SLpM:,Str. Acc.:,SApM:,Str. Def:,TD Avg.:,TD Acc.:,TD Def.:,Sub. Avg.:,Wins,Losses,Draws,Win Percentage
3,Shamil Abdurakhimov,Abrek,6.25,235,76,38.0,2.45,44,2.45,58,1.23,24,47,0.2,20,5,0,0.80
5,Daichi Abe,No Name,5.92,170,71,27.0,3.80,33,4.49,56,0.33,50,0,0.0,6,2,0,0.75
6,Papy Abedi,Makambo,5.92,185,74,41.0,2.80,55,3.15,48,3.47,57,50,1.3,9,3,0,0.75
7,Ricardo Abreu,Demente,5.92,185,73,35.0,3.79,31,3.98,68,2.13,42,100,0.7,5,3,0,0.62
8,Klidson Abreu,White Bear,6.00,205,74,26.0,1.97,39,1.93,59,0.50,12,75,0.0,15,3,0,0.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3381,Yao Zhikui,The Conqueror,5.42,125,64,28.0,1.44,21,2.82,48,0.66,16,58,0.0,2,5,0,0.29
3382,Fares Ziam,Smile Killer,6.08,155,75,22.0,0.67,32,1.40,57,0.00,0,75,0.0,10,3,0,0.77
3386,Cat Zingano,Alpha,5.50,145,68,37.0,2.57,61,1.63,47,2.77,65,42,0.8,10,4,0,0.71
3391,Allan Zuniga,Tigre,5.58,155,70,27.0,3.93,52,1.80,61,0.00,0,57,1.0,13,2,0,0.87


In [192]:
df.dtypes

Name               object
Nickname           object
Height:           float64
Weight:             int32
Reach:              int32
Age               float64
SLpM:             float64
Str. Acc.:         object
SApM:             float64
Str. Def:          object
TD Avg.:          float64
TD Acc.:           object
TD Def.:           object
Sub. Avg.:        float64
Wins                int32
Losses              int32
Draws               int32
Win Percentage    float64
dtype: object

In [193]:
df['Str. Acc.:'] = df['Str. Acc.:'].astype(float)
df['Str. Acc.:'] = df['Str. Acc.:']/100
df['Str. Def:'] = df['Str. Def:'].astype(float)
df['Str. Def:'] = df['Str. Def:']/100
df['TD Acc.:'] = df['TD Acc.:'].astype(float)
df['TD Acc.:'] = df['TD Acc.:']/100
df['TD Def.:'] = df['TD Def.:'].astype(float)
df['TD Def.:'] = df['TD Def.:']/100

In [194]:
df

,Name,Nickname,Height:,Weight:,Reach:,Age,SLpM:,Str. Acc.:,SApM:,Str. Def:,TD Avg.:,TD Acc.:,TD Def.:,Sub. Avg.:,Wins,Losses,Draws,Win Percentage
3,Shamil Abdurakhimov,Abrek,6.25,235,76,38.0,2.45,0.44,2.45,0.58,1.23,0.24,0.47,0.2,20,5,0,0.80
5,Daichi Abe,No Name,5.92,170,71,27.0,3.80,0.33,4.49,0.56,0.33,0.50,0.00,0.0,6,2,0,0.75
6,Papy Abedi,Makambo,5.92,185,74,41.0,2.80,0.55,3.15,0.48,3.47,0.57,0.50,1.3,9,3,0,0.75
7,Ricardo Abreu,Demente,5.92,185,73,35.0,3.79,0.31,3.98,0.68,2.13,0.42,1.00,0.7,5,3,0,0.62
8,Klidson Abreu,White Bear,6.00,205,74,26.0,1.97,0.39,1.93,0.59,0.50,0.12,0.75,0.0,15,3,0,0.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3381,Yao Zhikui,The Conqueror,5.42,125,64,28.0,1.44,0.21,2.82,0.48,0.66,0.16,0.58,0.0,2,5,0,0.29
3382,Fares Ziam,Smile Killer,6.08,155,75,22.0,0.67,0.32,1.40,0.57,0.00,0.00,0.75,0.0,10,3,0,0.77
3386,Cat Zingano,Alpha,5.50,145,68,37.0,2.57,0.61,1.63,0.47,2.77,0.65,0.42,0.8,10,4,0,0.71
3391,Allan Zuniga,Tigre,5.58,155,70,27.0,3.93,0.52,1.80,0.61,0.00,0.00,0.57,1.0,13,2,0,0.87


In [195]:
df.to_csv('plswork_V4.csv')